In [1]:

# Imports
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
import torchvision.transforms as transforms

from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!

torch.cuda.empty_cache()



In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
VGG_types = {
    "VGG11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG16": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        "M",
    ],
    "VGG19": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        512,
        "M",
    ],
}

In [5]:

#takes: channel x 224 x 224   image
class VGG(nn.Module):
    def __init__(self,architecture,in_channels=3,num_classes=10):
        super(VGG,self).__init__()
        self.in_channels=in_channels
        self.conv_filters=self.create_conv_layers(architecture)
        self.fc=nn.Sequential(

            nn.Linear(7*7*512,4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096,num_classes),
        )

    def forward(self,x):
        x=self.conv_filters(x)
        x= x.reshape(x.shape[0],-1)
        x=self.fc(x)
        return x
    def create_conv_layers(self,architecture):
        layers=[]
        in_channels=self.in_channels
        for x in architecture:
            
            if type(x) == int:
                layers+=[
                    nn.Conv2d(in_channels=in_channels,out_channels=x,kernel_size=(3,3),stride=(1,1),padding=(1,1)),
                    nn.BatchNorm2d(x),
                    nn.ReLU()
                ]
                in_channels=x

            elif x == 'M':
                layers+=[nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))]
        return nn.Sequential(*layers)
        

In [4]:
class VGG_net(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000):
        super(VGG_net, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(VGG_types["VGG16"])

        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x

    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            if type(x) == int:
                out_channels = x

                layers += [
                    nn.Conv2d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=(3, 3),
                        stride=(1, 1),
                        padding=(1, 1),
                    ),
                    nn.BatchNorm2d(x),
                    nn.ReLU(),
                ]
                in_channels = x
            elif x == "M":
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]

        return nn.Sequential(*layers)


In [6]:
model=VGG(architecture=VGG_types["VGG16"],in_channels=1,num_classes=10).to(device=device)
#model=VGG_net(in_channels=1,num_classes=10).to(device=device)


In [7]:
temp=torch.randn(64,1,224,224).to(device=device)

In [8]:
model(temp)

RuntimeError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 0; 3.82 GiB total capacity; 2.04 GiB already allocated; 336.56 MiB free; 2.05 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [9]:
temp=torch.randn(1,1,224,224).to(device=device)

print(model(temp).shape)


torch.Size([1, 10])


In [10]:
# Hyperparamters

num_classes=10
learning_rate=0.001
batch_size=1
num_epochs=16




In [11]:

#Loss and Optimizer

criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)


In [12]:
my_transforms=transforms.Compose([
    #transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])


In [13]:
# Loading dataset
train_dataset=datasets.MNIST(root='datasets/',train=True,transform=my_transforms,download=True)
train_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset=datasets.MNIST(root='datasets/',train=False,transform=my_transforms,download=True)
test_loader=DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)



In [14]:
def check_accuracy(loader,model):
    num_correct=0
    num_samples=0
    model.eval()  # to turn off batch normalisation and dropout layers

    with torch.no_grad():  # to turn off backprobagation
        for x, y in loader:
            x=x.to(device=device)
            #x=F.pad(input=x,pad=(2,2,2,2)).to(device=device) # make the image 32x32 dimension instead of 28x28

            y=torch.tensor(y).to(device=device)

            scores=model(x)
            _, predictions=scores.max(1)  #max return a tuple (max element, max element index)
            num_correct+= (predictions==y).sum()
            num_samples+= predictions.size(0)
        #print('accuracy=',((float(num_correct)/num_samples)*100) )

    model.train()   # to return model back to training mode
    return (float(num_correct)/num_samples)*100
    


In [15]:
# Train Network



for epoch in range(num_epochs):
    for batch_idx, (data,targets) in enumerate(tqdm(train_loader)):

        #getting the data
        data=data.to(device=device)
        targets=targets.to(device=device)
        # data is the batch of data, targets is the target label of each sample in the batch


        #forward
        scores=model(data)
        loss=criterion(scores,targets)
        
        torch.set_grad_enabled(True)  # Context-manager 

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent
        optimizer.step()
    #print('epoch_num=',epoch,'   Train accuracy=',check_accuracy(model=model,loader=train_dataset))
    #print('=======================================================')

        

#check_accuracy(model,test_dataset)



  0%|          | 0/60000 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 3.82 GiB total capacity; 2.12 GiB already allocated; 146.19 MiB free; 2.19 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [ ]:
print('test accuracy=',check_accuracy(loader=test_loader,model=model))

In [ ]:
torch.cuda.empty_cache()


In [ ]:
device

In [1]:
from torchsummary import summary


ModuleNotFoundError: No module named 'torchsummary'